In [1]:
import urllib.request

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2023):
    with open('sessionID') as f:
        sessionId = f.readlines()[0].strip()
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data
    

def parse_part(part):
    return eval(f'dict({part[1:-1]})')

def next_workflow(state, part, workflows):

    for workflow in workflows[state]:
        if ':' not in workflow:
            workflow = 'x>-9999:' + workflow

        op, st = workflow.split(':')
        variable = op[0]
        operation = op[1:]
        condition = eval(f'part["{variable}"] {operation}')
        
        if condition:
            if st == 'A' or st == 'R':
                return st
            else:
                return next_workflow(st, part, workflows)
    return None  # Or handle the case when no valid state is found


def next_workflow_part2(state, bounds):
    if state == 'A':
        return [bounds]

    if state == 'R':
        return []

    workflow = workflows[state]
    total = []

    for step in workflow:
        if ':' in step:
            cond, next = step.split(':')

            if '<'in cond:
                variable, high = cond.split('<')
                high = int(high)
                current = tighten(bounds, variable, 1, high)
                bounds = tighten(bounds, variable, high, maxvalue)
            else:
                variable, low = cond.split('>')
                low = int(low) + 1
                current = tighten(bounds, variable, low, maxvalue)
                bounds = tighten(bounds, variable, 1, low)

            total += next_workflow_part2(next, current)
        else:
            total += next_workflow_part2(step, bounds)

    return total
    
def tighten(bounds, variable, low, high):
    result = bounds.copy()
    result[variable] = (max(result[variable][0], low), min(result[variable][1], high))
    return result

import re
from itertools import groupby
from math import prod



In [2]:
[workflow_list, parts_list] = [list(y) for x, y in groupby(get_data(19), lambda z: z == '') if not x]
parts = list(map(parse_part, parts_list))


workflows = {}
for line in workflow_list:
    (state, instructions) =re.findall('(\\w+){(.*?)}', line)[0]
    workflows[state] = instructions.split(',')

accept = []
reject = []
for part in parts:
    if next_workflow('in', part, workflows) == 'A':
        accept.append(part)
part1 = 0
for state in accept:
    part1 += sum(state.values())
print(part1)


maxvalue = 4001
bounds = {variable: (1, maxvalue) for variable in 'xmas'}

part2 = 0
for line in next_workflow_part2('in', bounds):
    part2 += prod(hi - low for low, hi in line.values())
print(part2)

575412
126107942006821
